In [1]:
import tensorflow as tf
import json
from trainer import Trainer
from utils import map_word_to_vector
import numpy as np
import os

In [2]:
with open('107/checkpoints/model_configs.json', 'r') as inp:
    model_configs = json.load(inp)

In [3]:
with open('IntentDetection/full/word2idx.json', 'r') as inp:
    new_word2idx = json.load(inp)
with open('IntentDetection/full/char2idx.json', 'r') as inp:
    new_char2idx = json.load(inp)
with open('107/word2idx.json', 'r') as inp:
    old_word2idx = json.load(inp)
with open('107/char2idx.json', 'r') as inp:
    old_char2idx = json.load(inp)

In [4]:
with open('107/trainer_params.json', 'r') as inp:
    params = json.load(inp)

In [5]:
trainer = Trainer(**params)

In [6]:
trainer.build(save_optimizer_var=False)

INFO:tensorflow:Restoring parameters from 107/checkpoints/train/model.cpkt-73000


Restoring parameters from 107/checkpoints/train/model.cpkt-73000


In [7]:
model_configs['vocab_size'] = len(new_word2idx) + 1
model_configs['char_vocab_size'] = len(new_char2idx) + 1

In [8]:
tf.global_variables()

[<tf.Variable 'LanguageModel/decode_W:0' shape=(512, 122208) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/decode_b:0' shape=(122208,) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/embedding_weight:0' shape=(729, 16) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d/kernel:0' shape=(1, 16, 16) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d/bias:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d_1/kernel:0' shape=(2, 16, 16) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d_1/bias:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d_2/kernel:0' shape=(3, 16, 32) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d_2/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d_3/kernel:0' shape=(4, 16, 64) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d_3/bias:0' sh

In [9]:
with tf.variable_scope('LanguageModel', reuse=True):
    W_var = tf.get_variable('decode_W')
    b_var = tf.get_variable('decode_b')
    char_var = tf.get_variable('word_embedding/embedding_weight')
    W_ema_var = trainer.ema.average(W_var)
    b_ema_var = trainer.ema.average(b_var)
    char_ema_var = trainer.ema.average(char_var)

In [10]:
W_val = trainer.session.run(W_var)
b_val = trainer.session.run(b_var)
char_val = trainer.session.run(char_var)
W_ema_val = trainer.session.run(W_ema_var)
b_ema_val = trainer.session.run(b_ema_var)
char_ema_val = trainer.session.run(char_ema_var)

In [11]:
new_W_val = map_word_to_vector(new_word2idx, old_word2idx, W_val.T).T
new_b_val = np.squeeze(map_word_to_vector(new_word2idx, old_word2idx, np.expand_dims(b_val, axis=-1)), axis=-1)
new_char_val = map_word_to_vector(new_char2idx, old_char2idx, char_val)
new_W_ema_val = map_word_to_vector(new_word2idx, old_word2idx, W_ema_val.T).T
new_b_ema_val = np.squeeze(map_word_to_vector(new_word2idx, old_word2idx, np.expand_dims(b_ema_val, axis=-1)), axis=-1)
new_char_ema_val = map_word_to_vector(new_char2idx, old_char2idx, char_ema_val)

In [12]:
assign_W = tf.assign(W_var, new_W_val, validate_shape=False)
assign_b = tf.assign(b_var, new_b_val, validate_shape=False)
assign_char = tf.assign(char_var, new_char_val, validate_shape=False)
assign_W_ema = tf.assign(W_ema_var, new_W_ema_val, validate_shape=False)
assign_b_ema = tf.assign(b_ema_var, new_b_ema_val, validate_shape=False)
assign_char_ema = tf.assign(char_ema_var, new_char_ema_val, validate_shape=False)

In [13]:
trainer.session.run([assign_W, assign_b, assign_char, assign_W_ema, assign_b_ema, assign_char_ema])

[array([[-0.15977986, -0.2961595 , -0.09698444, ..., -0.05525641,
         -0.20132919, -0.25694156],
        [-0.08844721, -0.02871284,  0.08583482, ..., -0.00338753,
          0.01629108, -0.06739105],
        [ 0.09309864,  0.09625814,  0.07733241, ...,  0.11610243,
          0.05292799,  0.03755173],
        ...,
        [ 0.02139026,  0.05365834, -0.0212592 , ...,  0.1114517 ,
          0.10267484,  0.05574396],
        [ 0.09353878,  0.03639357,  0.0298811 , ...,  0.0905855 ,
          0.15521933, -0.00730767],
        [-0.06630649, -0.05841877, -0.0778296 , ..., -0.14346126,
         -0.14722826, -0.20143573]], dtype=float32),
 array([-10.655022 ,   2.2453089,   1.0510662, ...,  -6.962929 ,
         -9.498092 , -10.523586 ], dtype=float32),
 array([[ 0.08229961,  0.06513294, -0.06799582, ...,  0.07293753,
          0.01166529,  0.0735078 ],
        [-0.03917698, -0.06242321, -0.01078086, ...,  0.12380926,
         -0.01330725,  0.05882245],
        [ 0.04147386,  0.03022774, -0.

In [14]:
VERSION='107_intent'
trainer.checkpoint_dir = '{}/checkpoints'.format(VERSION)
trainer.model_configs = model_configs
trainer.save_configs()

In [15]:
trainer.train_saver.save(trainer.session, os.path.join(trainer.checkpoint_dir, 'train', 'model.cpkt'), global_step=trainer.session.run(trainer.global_step))

'107_intent/checkpoints/train/model.cpkt-73000'

In [16]:
trainer.test_saver.save(trainer.session, os.path.join(trainer.checkpoint_dir, 'test', 'model.cpkt'), global_step=trainer.session.run(trainer.global_step))

'107_intent/checkpoints/test/model.cpkt-73000'

In [17]:
params.update({
    'model_configs': model_configs,
    'log_path': '{}/logs'.format(VERSION),
    'train_summary_dir': '{}/train_summary/'.format(VERSION),
    'test_summary_dir': '{}/test_summary/'.format(VERSION),
    'checkpoint_dir': '{}/checkpoints/'.format(VERSION)
})

In [18]:
with open('{}/trainer_params.json'.format(VERSION), 'w') as out: json.dump(params, out)

In [19]:
params

{'model_configs': {'drop_e': 0.1,
  'skip_connection': True,
  'projection_dims': 512,
  'char_cnn_options': {'n_highways': 2,
   'layers': [[1, 16],
    [2, 16],
    [3, 32],
    [4, 64],
    [5, 128],
    [6, 256],
    [7, 512]]},
  'char_vocab_size': 1088,
  'vocab_size': 125409,
  'rnn_layers': [{'drop_o': 0.2, 'drop_i': 0.2, 'units': 1024, 'wdrop': 0.4},
   {'drop_o': 0.2, 'units': 1024, 'wdrop': 0.4},
   {'drop_o': 0.2, 'units': 1024, 'wdrop': 0.4}],
  'char_vec_size': 16},
 'bptt': 100,
 'optimizer': {'params': {'weight_decay': 1.2e-06,
   'beta2': 0.99,
   'beta1': 0.8},
  'name': 'adamw'},
 'train_summary_dir': '107_intent/train_summary/',
 'negative_samples': 10000,
 'wdecay': 0.0,
 'checkpoint_dir': '107_intent/checkpoints/',
 'log_path': '107_intent/logs',
 'beta': 1e-06,
 'use_ema': True,
 'test_summary_dir': '107_intent/test_summary/',
 'clip_norm': 0.3,
 'save_freq': 1000,
 'alpha': 1e-06}